In [99]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os
import Levenshtein

In [100]:
pd.options.display.max_rows = 400
pd.options.display.max_columns = 85

#### read in geocoded

In [101]:
df = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/new_totalprojects.csv')
df.head()

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
0,1,World,M,100.0,A,"1707 Edgerly Ave, Albany, Georgia, 31707","1707 Edgerly Ave, Albany, GA, 31707, USA",1707 Edgerly Ave,PointAddress,NaN,NaN,"1707 Edgerly Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1707.0,NaN,NaN,NaN,L,NaN,NaN,Edgerly,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1707 Edgerly Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,4208.0,USA,ENG,0.0,-84.191282,31.573277,-84.191292,31.573530,-84.192292,-84.190292,31.572530,31.574530,NaN,1707 Edgerly Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",-84.191293,31.573502,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,2,World,M,98.7,A,"703 Dartmouth Ln, Albany, Georgia, 31705","703 Dartmouth Ln, Albany, GA, 31705, USA",703 Dartmouth Ln,PointAddress,NaN,NaN,"703 Dartmouth Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,703.0,NaN,NaN,NaN,R,NaN,NaN,Dartmouth,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703 Dartmouth Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1211.0,USA,ENG,0.0,-84.136801,31.588103,-84.136802,31.588326,-84.137802,-84.135801,31.587326,31.589326,NaN,703 Dartmouth,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",-84.136794,31.588278,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,3,World,M,100.0,A,"1115 W Waddell Ave, Albany, Georgia, 31707","1115 W Waddell Ave, Albany, GA, 31707, USA",1115 W Waddell Ave,PointAddress,NaN,NaN,"1115 W Waddell Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1115.0,NaN,NaN,NaN,L,W,NaN,Waddell,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1115 W Waddell Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5115.0,USA,ENG,0.0,-84.178174,31.566717,-84.178184,31.567059,-84.179183,-84.177183,31.566059,31.568059,NaN,1115 W. Waddell Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",-84.178208,31.566964,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,4,World,M,98.7,A,"313 Forrestal Ln, Albany, Georgia, 31705","313 Forrestal Ln, Albany, GA, 31705, USA",313 Forrestal Ln,PointAddress,NaN,NaN,"313 Forrestal Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,313.0,NaN,NaN,NaN,L,NaN,NaN,Forrestal,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313 Forrestal Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,2508.0,USA,ENG,0.0,-84.082134,31.578470,-84.082442,31.578579,-84.083442,-84.081441,31.577579,31.579579,NaN,313 Forrestal,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",-84.082410,31.578511,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,5,World,M,100.0,A,"1125 Joel Dr, Albany, Georgia, 31707","1125 Joel Dr, Albany, GA, 31707, USA",1125 Joel Dr,PointAddress,NaN,NaN,"1125 Joel Dr, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1125.0,NaN,NaN,NaN,L,NaN,NaN,Joel,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1125 Joel Dr,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5021.0,USA,ENG,0.0,-84.207624,31.561222,-84.207901,31.561119,-84.208901,-84.206901,31.560119,31.562119,NaN,11

In [119]:
df[df['ShortLabel'].str.contains("Whispering Pines")]

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
69,70,World,M,98.70,A,"2514 Whispering Pines Rd, Albany, Georgia, 31707","2514 Whispering Pines Rd, Albany, GA, 31707, USA",2514 Whispering Pines Rd,PointAddress,NaN,NaN,"2514 Whispering Pines Rd, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,2514.0,NaN,NaN,NaN,R,NaN,NaN,Whispering Pines,Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2514 Whispering Pines Rd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,2234.0,USA,ENG,0.0,-84.210673,31.606356,-84.210673,31.606074,-84.211674,-84.209673,31.605074,31.607074,NaN,2514 Whispering Pines,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,00301/00005/040,2514 Whispering Pines,Albany,GA,"31.606147, -84.2106906",-84.210691,31.606147,2010,1,Homeowner Rehabilitation,828,Completed,HOME,48420.55,48420.55,0.0
159,160,World,M,100.00,A,"1404 Whispering Pines Rd, #Q2, Albany, Georgia...","1404 Whispering Pines Rd, #Q2, Albany, GA, 317...","1404 Whispering Pines Rd, #Q2",Subaddress,NaN,NaN,"1404 Whispering Pines Rd, #Q2, Albany, Georgia...",NaN,NaN,20.0,NaN,1404.0,NaN,NaN,NaN,R,NaN,NaN,Whispering Pines,Rd,NaN,NaN,NaN,NaN,NaN,Apt,Q2,#Q2,1404 Whispering Pines Rd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,2700.0,USA,ENG,0.0,-84.183683,31.604928,-84.183683,31.604616,-84.184682,-84.182682,31.603616,31.605616,NaN,1404 Whispering Pines Rd. Apt Q2,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000N/00027/08F,1404 Whispering Pines Rd. Apt Q2,Albany,GA,"31.6032603, -84.184194",-84.184194,31.603260,2013,16,Tenant Based Rental Assistance,1213,Completed,HOME,5095.00,5095.00,0.0
160,161,World,M,100.00,A,"1404 Whispering Pines Rd, Albany, Georgia, 31707","1404 Whispering Pines Rd, Albany, GA, 31707, USA",1404 Whispering Pines Rd,PointAddress,NaN,NaN,"1404 Whispering Pines Rd, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1404.0,NaN,NaN,NaN,R,NaN,NaN,Whispering Pines,Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1404 Whispering Pines Rd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,2700.0,USA,ENG,0.0,-84.183683,31.604928,-84.183683,31.604616,-84.184682,-84.182682,31.603616,31.605616,NaN,1404 Whispering Pines Rd. Apt. N-3,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000N/00027/08F,1404 Whispering Pines Rd. Apt. N-3,Albany,GA,"31.6032603, -84.184194",-84.184194,31.603260,2013,16,Tenant Based Rental Assistance,1177,Completed,HOME,9453.82,9453.82,0.0
213,214,World,M,100.00,A,"1404 Whispering Pines Rd, #B3, Albany, Georgia...","1404 Whispering Pines Rd, #B3, Albany, GA, 317...","1404 Whispering Pines Rd, #B3",Subaddress,NaN,NaN,"1404 Whispering Pines Rd, #B3, Albany, Georgia...",NaN,NaN,20.0,NaN,1404.0,NaN,NaN,NaN,R,NaN,NaN,Whispering Pines,Rd,NaN,NaN,NaN,NaN,NaN,Apt,B3,#B3,1404 Whispering Pines Rd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,2700.0,USA,ENG,0.0,-84.183683,31.604928,-84.183683,31.604616,-84.184682,-84.182682,31.603616,31.605616,NaN,1404 Whispering Pines Rd. Apt. B3,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000N/00027/08F,1404 Whispering Pines Rd. Apt. B3,Albany,GA,"31.6032603, -84.184194",-84.184194,31.603260,2015,10,Tenant Based Rental Assistance,1326,Completed,HOME,11986.83,11986.83,0.0
214,215,World,M,100.00,A,"1404 Whispering Pines Rd, #N2, Albany, Georgia...","1404 Whispering Pines Rd, #N2, A

#### read in originals

In [102]:
xlsx = pd.ExcelFile('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)
df_original.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 Joel Drive,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


#### fix new

In [103]:
df1_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/HOME_2007-2017_fixed_address.csv')
df2_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/CDBG_2007-2017_fixed_address.csv')

In [104]:
df1_fixed = df1_fixed.drop('Unnamed: 0', axis = 1)
df2_fixed = df2_fixed.drop('Unnamed: 0', axis = 1)

In [105]:
series_funded = df1_fixed['Funded Amount']
series_funded

0       27033.46
1       30185.60
2       28678.19
3       34694.47
4       30756.96
5       61252.07
6       29927.62
7       28778.79
8       37189.39
9       30159.70
10      29884.52
11      31324.48
12      30396.28
13      19017.18
14      19017.18
15      19017.18
16      19017.18
17      19017.18
18      19017.18
19      19017.18
20      19017.18
21      19017.18
22      19017.19
23      19017.19
24      32802.78
25      39317.93
26     139075.23
27     136412.23
28      40580.36
29      39592.06
30      34193.74
31      43147.02
32      29571.17
33       4186.88
34       3444.04
35      38242.07
36      43492.75
37     138941.45
38      16407.64
39      50590.82
40     110554.62
41       3638.20
42      19223.15
43      74097.76
44      40834.32
45      96315.89
46      53194.73
47      44850.56
48      48700.00
49       2161.56
50     134942.90
51     134869.84
52      33423.30
53      66618.44
54      99879.78
55      45902.14
56     118817.44
57      53553.46
58      33081.

In [106]:
df1_fixed.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [107]:
df2_fixed.columns

Index(['Parcel ID', 'Address', 'City', 'State', 'XY Coordinates', 'Plan Year',
       'IDIS Project', 'Project', 'IDIS Activity ID', 'Activity Status',
       'Program', 'Funded Amount', 'Draw Amount', 'Balance'],
      dtype='object')

#### comparisons

In [108]:
df1 = df[df['Program'] == "HOME"]
df2 = df[df['Program'] == "CDBG"]

In [109]:
df_utilities2012 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]

In [13]:
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
series_utilities = pd.Series(list2)

In [118]:
series_utilities = df_utilities2012['Premise Address']

In [111]:
series_utilities

0                 300  ACORN  ST
1                 300  ACORN  ST
2                 300  ACORN  ST
3                 300  ACORN  ST
4                 212  ACORN  ST
5                 212  ACORN  ST
6                 212  ACORN  ST
7                 212  ACORN  ST
8                 212  ACORN  ST
9                 212  ACORN  ST
10                212  ACORN  ST
11            2101  LEONARD  AVE
12            2101  LEONARD  AVE
13            2101  LEONARD  AVE
14            2101  LEONARD  AVE
15            2101  LEONARD  AVE
16            2101  LEONARD  AVE
17            2101  LEONARD  AVE
18            2103  LEONARD  AVE
19            2103  LEONARD  AVE
20            2103  LEONARD  AVE
21            2103  LEONARD  AVE
22            2103  LEONARD  AVE
23            2103  LEONARD  AVE
24            2103  LEONARD  AVE
25           636  JEFFERIES  AVE
26           636  JEFFERIES  AVE
27           636  JEFFERIES  AVE
28           636  JEFFERIES  AVE
29           636  JEFFERIES  AVE
30        

In [112]:
series_housing = df1_fixed['Address']

In [113]:
addresses = []
for address in series_housing:
    address = address.strip()
    new_address = ""
    address_split = address.split(' ')
    length = len(address_split)
    for i in range(len(address_split)):
        if i == 1:
            if not (address_split[i] == 'N' or address_split[i] == 'S'
            or address_split[i] == 'E' or address_split[i] == 'W'):    
                new_address += ' '
        if i == length-1:
            new_address += ' '
        new_address += (address_split[i] + ' ')
    new_address = new_address.strip()
    addresses.append(new_address)
series_housing = pd.Series(addresses)

In [114]:
series_housing

0                        1707  EDGERLY  AVE
1                        703  DARTMOUTH  LN
2                       1115 W WADDELL  AVE
3                        313  FORRESTAL  LN
4                            1125  JOEL  DR
5                          1519  MALONE  DR
6                          719  ANDOVER  LN
7                      305 S SHADOWLAWN  DR
8                           1662  SUNNY  LN
9                         511  MOULTRIE  RD
10                          1100  SWIFT  ST
11                    1009 E ROOSEVELT  AVE
12                       1118  WHITNEY  AVE
13                           431  EBONY  LN
14                           529  EBONY  LN
15                           535  EBONY  LN
16                           539  EBONY  LN
17                           545  EBONY  LN
18                           559  EBONY  LN
19                           563  EBONY  LN
20                           531  EBONY  LN
21                           537  EBONY  LN
22                           543

In [115]:
series_unmatched = series_housing[~(series_housing.isin(series_utilities.str.upper()))]

12                       1118  WHITNEY  AVE
24                     1403  GREENBRIAR  CT
28                       1602  ESTELLE  AVE
57                       2620 S MADISON  ST
66                         1330  MOBILE  RD
71                        504  MAGNOLIA  ST
73                       1110  WADDELL  AVE
74                          1605  SUNNY  LN
80                    2700  A DONCASTER  DR
81                    2701  A DONCASTER  DR
84                        529  COCHRAN  AVE
86                 2226  B OLD DOMINION  DR
87                    2701  B DONCASTER  RD
88                     250  BONNY VIEW  AVE
89                     249  BONNY VIEW  AVE
90                     248  BONNY VIEW  AVE
91                     251  BONNY VIEW  AVE
92                     253  BONNY VIEW  AVE
93                     247  BONNY VIEW  AVE
94                     254  BONNY VIEW  AVE
95                     243  BONNY VIEW  AVE
96                     245  BONNY VIEW  AVE
97                     252  BONN

In [130]:
temp_list = []
temp_list_u = []
address = "200 S VALENCIA  DR"
for u_address in series_utilities:
    if Levenshtein.distance(address, u_address.upper()) <= 3:
        temp_list.append(address)
        temp_list_u.append(u_address.upper())
temp_list_series = pd.Series(temp_list)
temp_list_u_series = pd.Series(temp_list_u)

In [131]:
temp_list_u_series

0      507 S VALENCIA  DR
1      507 S VALENCIA  DR
2      507 S VALENCIA  DR
3      507 S VALENCIA  DR
4      507 S VALENCIA  DR
5      207 S VALENCIA  ST
6      207 S VALENCIA  ST
7      207 S VALENCIA  ST
8      207 S VALENCIA  ST
9      207 S VALENCIA  ST
10     205 S VALENCIA  ST
11     205 S VALENCIA  ST
12     205 S VALENCIA  ST
13     205 S VALENCIA  ST
14     205 S VALENCIA  ST
15     205 S VALENCIA  ST
16     203 S VALENCIA  ST
17     203 S VALENCIA  ST
18     203 S VALENCIA  ST
19     203 S VALENCIA  ST
20     203 S VALENCIA  ST
21     203 S VALENCIA  ST
22     201 S VALENCIA  ST
23     201 S VALENCIA  ST
24     201 S VALENCIA  ST
25     201 S VALENCIA  ST
26     201 S VALENCIA  ST
27     201 S VALENCIA  ST
28     200 S VALENCIA  ST
29     200 S VALENCIA  ST
30     200 S VALENCIA  ST
31     200 S VALENCIA  ST
32     200 S VALENCIA  ST
33     200 S VALENCIA  ST
34     202 S VALENCIA  ST
35     202 S VALENCIA  ST
36     202 S VALENCIA  ST
37     202 S VALENCIA  ST
38     202 S

seems like when they add a cardinal direction, there is only one space between number, dir, and streetname, but still 2 between street and street type <br>
1118 whitney ave is not found in utilities -- fixed: should be w whitney<br>
greeenbrier ct in utilities not greenbriar <br>
estelle st in utilities not ave <br>
2620 s madison is missing st in utilities data
1330 mobile road seems to be missing, there are mobile road addresses however --fixed should be avenue<br>
same with 504 magnolia st --fixed should be 504 s magnolia street <br>
1110 waddell should be west waddell <br>
520 cochran in utilities list but not 529 <br>
unclear about this but bonnyview is one word on google maps but two on the actual street sign. <br>
200s for Bonny view doesn't really seem to be in utilities data? <br>
apts are not listed in utilities data <br>
independence dr not ave in utilities data <br>
dawson rd r1 meaning? no r1 in utilities <br>
same with don cutler sr dr 90?. <br>
should be west highland ave <br>
typo in utilities should be 603 n haley st <br>
1112 peachtree ter not 1112a <br>
305 station crossing should be station crossing dr <br>
1902 south flintlock dr <br>
818 westview drive? does it exist? <br>
farrell ct not ferrell <br>
1003 holland dr is not existing yet? <br>
mclain not mcclain <br>
amsterdam ln not dr <br>
valencia st not dr <br>

In [20]:
Levenshtein.distance('hi', "hey")

2

In [56]:
temp_list = []
temp_list_u = []

counter = 0
for address in series_housing:
    if counter % 20 == 0:
        print(str(counter))
    for u_address in series_utilities:
        if Levenshtein.distance(address, u_address) < 4:
            if Levenshtein.distance(address, u_address) != 0:         
                temp_list.append(address)
                temp_list_u.append(u_address)
    counter += 1
temp_list_series = pd.Series(temp_list)
temp_list_u_series = pd.Series(temp_list_u)

0


KeyboardInterrupt: 